# Other aspects of Q
Notes on various aspects in q

### Array processing Lanuge:
Allows hardware to opperate on an entire array at once. As opposed to at a scaler level in cpu. Many modern languges have extentions to operate on an array, like numpy for Python.

Use case:

well suited for `implicite parallelization`. Note: not parallel as in parallel porcessing, since a single processor is used to operate on a list.

Example:

In [27]:
p)# summing two arrays pairwise in python
p)a = [1,2,3]
p)b = [4,5,6]
p)c = [x+y for x,y in zip(a,b)]
p)print(c)

[5, 7, 9]


In [28]:
// summing in q
a:1 2 3;
b:4 5 6;
// implicite parallelization
show c:a+b

5 7 9


Try running the fibonacchi funciton using recursion vs scan/over and time the difference. Which is more efficent? Why?

Try running Factorial as well

In [29]:
// fibonacci sequence with recursion
fib:{$[x~0; :0; x~1; :1; :fib[x-1]+fib[x-2]]};
fib each 1 + til 5
\ts fib each 1 + til 30

1 1 2 3 5


1480 4256


In [30]:
// fibonacci sequence with over
show 5 {: x, sum -2#x}/ 0 1;
\ts 30 {: x, sum -2#x}/ 0 1

0 1 1 2 3 5 8


0 1952


In [31]:
// factorial with with recursion
show {$[x~1; :1 ; : x * .z.s[x-1]]} 5;
\ts {$[x~1; :1 ; : x * .z.s[x-1]]} 30

120


0 3072


In [32]:
// factorial with over
show {x */ desc 1+ til x-1}5;
\ts {x */ desc 1+ til x-1} 30

120


0 2176


We can see that over is exponentially faster since it takes advantage of q's array processing ability. Although the factorial funciton is fast in both it's memory complexity is less with over

### Attributes
different attributes, why and how to apply to table.

The 4 attributes:
- \`#s - sorted : 
    - Applied to lists columns that are sorted in ascending order. There in no memory overhead. 
    - Why? It allows for binary search to improve the search time of query.
- \`#u - unique : 
    - Applied to group of distinct items such as key column of a keyed table or domain of a dictionary. It creates an internal hashmap (allows it to find index in o(1) time) which uses a large amount of memory. 
    - Why? Speeds up searches like distinct and can end searches faster since it knows to only look for one occurance.
- \`#p - parted : 
    - Applied on lists when the items are continuous but not in ascending or decending order. It's usually applied to the sym column in HDB. There is a small memory overhead. 
    - Why? It creates an internal dictionary mapping each output to the first occurrance. Which then allows it to find all occurance quickly since it's located continuously, thereby speeding up the query.
- \`#g - grouped : 
    - Applied on lists with no apperent structure. Used like indexing in most databases. creates a dictionary withg each distinct occurance as a key and the indexes of that occurance as a value. The dictionary will use large amount of memory. 
    - Why? Greatly improves speed since it can search the duictionary instead of each item.


Memory Overhead:
 - Grouped (largest)
 - Unique
 - Parted (Smallest)
 - Sorted (No memory overhead)
 
- Which attribute will you find in HDB and RDB respectivly?
    - HDB syms are parted
    - RDB time is sorted
    
- What attribute would you place in a trade and quote table?
    - sorted by time to maintain order of events
    - grouped on sym if speed is of greater concern than memory and the table isn't appeneded to
    - perhaps grouped on exchange if that's often of concern

In [33]:
// setting up table for examples
t1:([]syms:500?`AAPL`BABA`CAT`DELTA`EY;prices:500?1000.00)
show meta t1;
t1

c     | t f a
------| -----
syms  | s    
prices| f    


syms  prices  
--------------
AAPL  347.9384
DELTA 159.7991
DELTA 474.2426
BABA  821.5712
BABA  656.6956
DELTA 728.6657
AAPL  941.5102
CAT   643.3012
CAT   514.6122
CAT   708.6662
DELTA 887.5002
BABA  274.4299
BABA  878.2214
EY    55.36874
EY    58.1007 
AAPL  568.2217
DELTA 69.48637
EY    677.7274
EY    637.2055
EY    724.9415
..


In [34]:
// set sorted attribute to sym column in table
meta update syms:`s#asc syms from t1  // note sort is implicite (didnt need the `s#)

c     | t f a
------| -----
syms  | s   s
prices| f    


In [35]:
meta select syms:`u#syms, prices from t1 where i=(first;i) fby syms // needed to make unique to apply attributute

c     | t f a
------| -----
syms  | s   u
prices| f    


In [36]:
meta select syms:`p#syms, prices from `syms`prices xasc t1

c     | t f a
------| -----
syms  | s   p
prices| f    


In [37]:
meta update syms:`g#syms from t1 

c     | t f a
------| -----
syms  | s   g
prices| f    


- What is an enumeration? What is the use case?
> Enumeration is a list mapped to a distinct list that contains its values.iT Allows us to reduce storage space by creating a map from the distinct list to its occurances. 

In [38]:
// example of enumeration
distLst:`AAPL`BABA`GOOG`AMZN`TSLA;
show lst:20?distLst;
show lst2:`distLst$lst;

`GOOG`BABA`AMZN`AAPL`BABA`TSLA`BABA`AAPL`AMZN`GOOG`AMZN`AAPL`AMZN`AAPL`TSLA`A..
`distLst$`GOOG`BABA`AMZN`AAPL`BABA`TSLA`BABA`AAPL`AMZN`GOOG`AMZN`AAPL`AMZN`AA..


In [39]:
`distLst!til 5

`distLst$`AAPL`BABA`GOOG`AMZN`TSLA


In [40]:
// Extend enumeration
show `distLst?lst2, 5?`NFLX`FB;
`distLst!til 7

`distLst$`GOOG`BABA`AMZN`AAPL`BABA`TSLA`BABA`AAPL`AMZN`GOOG`AMZN`AAPL`AMZN`AA..


`distLst$`AAPL`BABA`GOOG`AMZN`TSLA`FB`NFLX


In [41]:
-10#lst2,: 5?`NFLX`FB

`distLst$`AMZN`GOOG`AMZN`TSLA`TSLA`NFLX`FB`FB`FB`NFLX


In [42]:
distLst

`AAPL`BABA`GOOG`AMZN`TSLA`FB`NFLX


### Joins

The 10 joins:
- , : simple join
- upsert : upsert
- ij : inner join
- lj : left join
- uj : union join
- ej : equal join
- pj : plus join
- aj : As of join;
- asof : Second time join.
- wj : Third Time join. 


- Joins where t2 must be keyed?
    - ij
    - lj
    - pj

- Joins where t2 must NOT be keyed?
    - uj
    - aj

    
- Whats the difference between aj and aj0?
    - aj is used for getting the time in the first table while aj0 uses the time in the second table.

- Whats the difference between lj and ljf?
    - returns values from t1 if not present in t2, as of v3.6 its the same as lj.

In [43]:
// Setting up tables for joins
t1:([]a:1 2 3 4;b:10 20 30 45)
t2:([]a:4 5 6;b:40 50 60)
kt1:([a:1 2 3 4];b:10 20 30 45)
kt2:([a:4 5 6];b:40 50 60)

#### - , : simple join
    - t1 , t2
    - When using , the column names must match. The column types do not have to match. The result when using on unkeyed tables is simmilar to insert. When using on keyed table the result is simmilar to upsert where the values in t2 will replace the values in the same key if that key is present else it will add in a new key with the values from t2.

In [44]:
show t1,t2;
`t1 insert t2;
show t1;
// cleanup
t1:-3_t1;

a b 
----
1 10
2 20
3 30
4 45
4 40
5 50
6 60
a b 
----
1 10
2 20
3 30
4 45
4 40
5 50
6 60


In [45]:
show kt1 upsert kt2;
(kt1, kt2) ~ (kt1 upsert kt2)

a| b 
-| --
1| 10
2| 20
3| 30
4| 40
5| 50
6| 60


1b


#### - upsert : upsert
    - Simmilar to , (simple join). It adds in values from t1 to t2. When t1 is keyed it replaces the key's value with t2 when t1 is unkeyed it adds in the the values from t2. If t1 is unkeyed so must t2.

In [46]:
t1 upsert t2 // adds value like insert

a b 
----
1 10
2 20
3 30
4 45
4 40
5 50
6 60


In [47]:
kt1 upsert kt2 // upserts value replaces key

a| b 
-| --
1| 10
2| 20
3| 30
4| 40
5| 50
6| 60


#### - ij : inner join
    - t1 ij t2
    - Joins the rows in t1 with t2 for values present in the keys in t2. The columns in t1 must contain the keyed columns in t2, and t2 must be keyed. Will return the rows in t1 which are present in column of t2.

In [48]:
show t1:([]a:1 2 3;b:10 20 30;c:100 200 300);
show kt2:([a:1 2 4]c:-100 0N -400; d:-100 -2000 -4000);
t1 ij kt2

a b  c  
--------
1 10 100
2 20 200
3 30 300
a| c    d    
-| ----------
1| -100 -100 
2|      -2000
4| -400 -4000


a b  c    d    
---------------
1 10 -100 -100 
2 20      -2000


In [49]:
(1!t1) ij kt2  // t1 may or maynot be keyed

a| b  c    d    
-| -------------
1| 10 -100 -100 
2| 20      -2000


#### - lj : left join
    - t1 lj t2
    - Joins the columns of t1 with the columns in t2 for rows in t2 which must be keyed and have matching key column in t1. The difference from this and ij is it returns all the columns in t1 and adds in the values from t2 for rows that match on the keys. For the unmatched rows it returns.

In [50]:
t1 lj kt2

a b  c    d    
---------------
1 10 -100 -100 
2 20      -2000
3 30 300       


#### - uj : union join
    - t1 uj t2
    - Joins two table vertically, simmilar to a full outter join. Where t1 may or may not be keyed and t2 must not be keyed.The column names doesn't need to match. Common usecase for this is to join trade and quote and sort time to get a clear picture of how quotes lead to trade.

In [51]:
t1:([]a:1 2;b:2 3;c:5 7);
t2:([]a:1 2 3;b:2 3 7;c:10 20 30;d:"ABC");
t1 uj t2

a b c  d
--------
1 2 5   
2 3 7   
1 2 10 A
2 3 20 B
3 7 30 C


#### - ej : equal join
    - ej[c;t1;t2]
    - Joins two tables based on matching column(s) c. If multiple values are present the rows are repeted.

In [52]:
t1:([]sym:`IBM`FDP`FDP`FDP`IBM`MSFT;price:0.7029677 0.08378167 0.06046216 0.658985 0.2608152 0.5433888)
t2:([]sym:`IBM`MSFT`FDP;ex:`N`CME`O;MC:1000 250 300)
ej[`sym;t1;t2]

sym  price      ex  MC  
------------------------
IBM  0.7029677  N   1000
FDP  0.08378167 O   300 
FDP  0.06046216 O   300 
FDP  0.658985   O   300 
IBM  0.2608152  N   1000
MSFT 0.5433888  CME 250 


In [53]:
ej[`sym;t2;t1]

sym  ex  MC   price     
------------------------
IBM  N   1000 0.7029677 
IBM  N   1000 0.2608152 
MSFT CME 250  0.5433888 
FDP  O   300  0.08378167
FDP  O   300  0.06046216
FDP  O   300  0.658985  


#### - pj : plus join
    - t1 pj t2
    - Simmilar to an left join, it joins t1 with the keyed columns in t2 where t2 must be keyed, and returns row for every row in t1. adds the numeric values of the common cols other than the keyed columns. If no matching keys are in t2 it leaves the values un-added.
    - Might be useful for joining quantities of stocks bought tracked in different tables

In [54]:
t1:([]a:1 2 5;b:2 3 7;c:5 7 11);
kt2:([a:1 2 3];b:2 3 7;c:10 20 30;d: 100 200 300);
t1 pj kt2

a b c  d  
----------
1 4 15 100
2 6 27 200
5 7 11 0  


####  - aj : As of join
    - aj[`col1...`coln`time;t1;t2]
    - Simmilar to a left join it returns the last in time values from t2.
    - First of the time joins. This join is for joining table with reference to time. It is used for getting the values of t2 that occured most recently respective to the time value in t1. 
    - In here t2 must not be keyed.
    - The constraint columns c1 to cn are used to match the values in both table so if its present it needs to be in both table. The cols c1 to cn does not need to be specified, but the time column is required in both tables. The column order c1 to cn should be applied to reduce the search space in order. Where c1 narrows down the search space the most and c2 the second most and so on.
    - This query is mainly used to find the prevailing quote for a trade, trade being t1 and quote t2.
    
    - aj0 - returns time from t2 whereas aj returns time from t1
    

In [55]:
t1:([]time:10:01:01 10:01:03 10:01:04;sym:`msft`ibm`ge;qty:100 200 150)
t2::([]time:10:01:00 10:01:00 10:01:00 10:01:02;sym:`ibm`msft`msft`ibm;px:100 99 101 98)
aj[`sym`time;t1;t2]

time     sym  qty px 
---------------------
10:01:01 msft 100 101
10:01:03 ibm  200 98 
10:01:04 ge   150    


In [56]:
aj0[`sym`time;t1;t2]

time     sym  qty px 
---------------------
10:01:00 msft 100 101
10:01:02 ibm  200 98 
10:01:04 ge   150    


#### - asof : Second time join.
    - asof[t1,t2 or d2]
    - Joins t1 is a table with a time column, t2 is a table or dictionary with the last key or column as a time column. It returns the colums in t1 (as a dict or table based on input) thats last in time to t2/d2 with the matching keys (which are the keys or cols in d2/t2)  
    - t2 or d2 must have all the cols in t1

In [57]:
t1:([]sym:`AAPL`IBM;time:09:28:00.0 09:29:00.0;ex:"TD";price: 78.14 96.30;size:100 200)
t1 asof`sym`time!(`IBM;09:30:00.0)

ex   | "D"
price| 96.3
size | 200


In [60]:
// only ibm trade is asof the times listed in this table
t1 asof([]sym:`AAPL`IBM;ex:"TD";time:09:10:00.0 09:30:00.0)

price size
----------
          
96.3  200 


#### - wj : Third Time join. 
    - wj[w; c; t1; t2] or wj[w; c; t1; (t2;(f0;c0);(f1;c1))]
    - Join that returns for each record in t1 with values in t2 where the records are present in the time interval w.
    - t1 and t2 are simple table (in real usecases t1 is trade and t2 is quote)
    - c is a set of columns ending in time (cn=`time) as the last one, time is the only col thats required. c1..cn-1 should be ordered to decrease the rows the most.
    - (f0;c0) is an aggregate function to be applied to a column in t2

In [61]:
t:([]sym:3#`ibm;time:10:01:01 10:01:04 10:01:08;price:100 101 105)
q:([]sym:9#`ibm; time:10:01:01+til 9; ask:101 103 103 104 104 107 108 107 108; bid:98 99 102 103 103 104 106 106 107)

In [62]:
wj[-2 1+\:t.time;`sym`time;t;(q;(max;`ask);(min;`bid))]

sym time     price ask bid
--------------------------
ibm 10:01:01 100   103 98 
ibm 10:01:04 101   104 99 
ibm 10:01:08 105   108 104


### q-Sql vs Funcitonal Queries

Functional Statements
 - For select or exec
 `?[t;c;b;a]`
 - for update or delete
 `![t;c;b;a]`
 - optional
 `?[t;c;b;a;n;(g;cn)]`
 
 - t : is the table 
 - c : is a list of where clause constraints
 - b : dictionary of grouping constraints
 - a : dictionary of aggregates
 --- optional:
 - n the number of records or a window of records
 - (g;cn) g is a uninary grading funciton (example idesc - decending order) and cn is the column name it'll apply to
 
- Whats the difference between select and exec?
    - When attempting to exec for b, the grouping funciton, you would use an atom value 0b instead of () used for select.
 
- Whats the difference between update and delete?
    - When deleting the a becomes a simple list of column names, not a dictionary of aggregates. When deleting either a or c must be present not both, since it does not depete both columns and rows at the same time.
    

### Data in disk

- Splayed 
- partitioned
- segmented


- .Q.en
- .Q.dfpt
- .Q.hdpf
- .Q.dsftg

### Timer and timer table
- .z.ts opperates on the time set by \t
- we can use this to set a whole host of funcitons that opperate at intervals

### Ticker Plant

- What are the best practices when replaying a ticker plant log file?
    - -11!`:/handle/To/LogFile
        - same as q)value each get `:/handle/To/LogFile but uses far less memory
        - if successful the number of entries is returnerd
        - if not, will error with 'badtail
     - if table or function is missing it'll display the name ie 'upd 'trade
     - -11(-2;`:/handle/To/LogFile) returns the number of chuncks in valid tp file else it returns the number of valid chunks

- Name 6 common ticker plant functions?
 - .u.pub - 
 - .u.sub - 
 - .u.upd - 
 - .u.w - dictionary of tables->(handle;syms)
 - .u.end - save data in disk, clear and reload hdb
 - 

In [63]:
// to show examples for Ticker Plant

### .z and .Q name space interesting funcitons

- .z.pg
- .z.ps
- .z.W

- .Q.ft
- .Q.